In [258]:
from pydantic import BaseModel

class Person(BaseModel):
    first_name: str
    last_name: str
    age: int

In [259]:
# Create model instance
person = Person(first_name="John", last_name="Doe", age=35)

In [260]:
str(person)

"first_name='John' last_name='Doe' age=35"

In [261]:
# Representation of the model
repr(person)

"Person(first_name='John', last_name='Doe', age=35)"

In [262]:
# inspect fields
person.model_fields

{'first_name': FieldInfo(annotation=str, required=True),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=True)}

In [263]:
from pydantic import BaseModel

class Person(BaseModel):
    first_name: str
    last_name: str
    age: int

    @property
    def display_name(self) -> str:
        return f"{self.first_name} {self.last_name}"

person = Person(first_name="John", last_name="Doe", age=35)

print(person.display_name)

John Doe


In [264]:
# By default, Pydantic will only validate the fields when the model instance is created.
from pydantic import ValidationError

try:
    Person(first_name="John", last_name="Doe", age="twenty")
except ValidationError as ex:
    print(ex)

# It will not raise an error.
person.age = "twenty"

print(person)


1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='twenty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing
first_name='John' last_name='Doe' age='twenty'


# Deserialization

In [265]:
data = {
    "first_name": "John",
    "last_name": "Doe",
    "age": 20
}

_Not recommended method_

for simples models a dictionary unpacking can be used to deserialize the object. This method is not recommended for complex models.

In [266]:
# Create a model instance from a dictionary
Person(**data)

Person(first_name='John', last_name='Doe', age=20)

## Deserialization from dictionary

Use this instead:

`Model.model_validate(dict_data)`

In [267]:
Person.model_validate(data)

Person(first_name='John', last_name='Doe', age=20)

## Deserialization from JSON

In [268]:
data_json = """
{
    "first_name": "John",
    "last_name": "Doe",
    "age": 20
}
"""

In [269]:
Person.model_validate_json(data_json)

Person(first_name='John', last_name='Doe', age=20)

# Serialization

In [270]:
newton = Person(first_name="Issac", last_name="Newton", age=84)
tesla = Person(first_name="Nikola", last_name="Tesla", age=86)

In [271]:
# This serializes the model instance to a dictionary
print(tesla.model_dump())

# Type of the serialized dictionary
print(type(tesla.model_dump()))

{'first_name': 'Nikola', 'last_name': 'Tesla', 'age': 86}
<class 'dict'>


In [272]:
# This serializes the model instance to a JSON string
print(type(newton.model_dump_json()))
print(newton.model_dump_json())

# Custom indentation
print(newton.model_dump_json(indent=4))

<class 'str'>
{"first_name":"Issac","last_name":"Newton","age":84}
{
    "first_name": "Issac",
    "last_name": "Newton",
    "age": 84
}


## Control how to the data is serialized

In [273]:
# Exclude fields
print(newton.model_dump_json(indent=4, exclude={"age", "first_name"}))

{
    "last_name": "Newton"
}


In [274]:
# Include fields
print(newton.model_dump_json(indent=4, include={"age"}))

{
    "age": 84
}


# Type Coercion
> _Type Coercion refers to the process of automatic or implicit conversion of values from one data type to another._

__See:__ https://docs.pydantic.dev/latest/concepts/conversion_table/

In [275]:
class Coordinates(BaseModel):
    x: float
    y: float

In [276]:
p1 = Coordinates(x=1.0, y=2.0)

In [277]:
p1.model_fields

{'x': FieldInfo(annotation=float, required=True),
 'y': FieldInfo(annotation=float, required=True)}

In [278]:
# Pydanitc will convert the integer to a float and the string to a float
p2 = Coordinates(x=1, y="2.3")
display(p2)
print(type(p2.x), type(p2.y))

Coordinates(x=1.0, y=2.3)

<class 'float'> <class 'float'>


In [279]:
class ConversionModel(BaseModel):
    field: str

In [280]:
try:
    ConversionModel(field=1)
except ValidationError as ex:
    print(ex)

1 validation error for ConversionModel
field
  Input should be a valid string [type=string_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type


# Required vs Optional fields

In [281]:
# This has 2 required fields
class Circle(BaseModel):
    center: tuple[int, int]
    radius: int

Circle.model_fields

{'center': FieldInfo(annotation=tuple[int, int], required=True),
 'radius': FieldInfo(annotation=int, required=True)}

In [282]:
# This has 1 Optional field and 1 required field
class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int

Circle.model_fields

{'center': FieldInfo(annotation=tuple[int, int], required=False, default=(0, 0)),
 'radius': FieldInfo(annotation=int, required=True)}

In [283]:
Circle(radius=10)

Circle(center=(0, 0), radius=10)

In [284]:
circle_data_dict = {"radius": 10}
circle_data_json = '{"radius": 10}'

display(Circle.model_validate_json(circle_data_json))
display(Circle.model_validate(circle_data_dict))

circle_data_dict = {"radius": 10, "center": (4, 2)}
display(Circle.model_validate(circle_data_dict))


Circle(center=(0, 0), radius=10)

Circle(center=(0, 0), radius=10)

Circle(center=(4, 2), radius=10)

# Nullable fields

In [285]:
class NullModel(BaseModel):
    field: int

In [286]:
# Invalid value
try:
    NullModel(field=None)
except ValidationError as ex:
    print(ex)

1 validation error for NullModel
field
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.8/v/int_type


In [287]:
# Required field
try:
    NullModel()
except ValidationError as ex:
    print(ex)

1 validation error for NullModel
field
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing


In [288]:
class NullModel(BaseModel):
    field: int | None # Nullable field

In [289]:
# Model with a nullable field
display(NullModel(field=None))

# The filed still required
try:
    NullModel()
except ValidationError as ex:
    print(ex)

NullModel(field=None)

1 validation error for NullModel
field
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing


In [290]:
class NullModel(BaseModel):
    field: int | None = None # Nullable field with a default value

In [291]:
# Model with a nullable field
display(NullModel(field=None))

# The filed is not required
display(NullModel())


NullModel(field=None)

NullModel(field=None)

## Combination of Optional and Nullable

### Required, not nullable

In [292]:
# Default
class TestModel(BaseModel):
    field: int

# Required field
try:
    TestModel()
except ValidationError as ex:
    print(ex)

# Not nullable field
try:
    TestModel(field=None)
except ValidationError as ex:
    print(ex)

1 validation error for TestModel
field
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
1 validation error for TestModel
field
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.8/v/int_type


### Required, nullable

In [293]:
class TestModel(BaseModel):
    field: int | None

# Required field
try:
    TestModel()
except ValidationError as ex:
    print(ex)

# nullable field
try:
    display(TestModel(field=None))
except ValidationError as ex:
    print(ex)

1 validation error for TestModel
field
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing


TestModel(field=None)

### Optional, not nullable

In [294]:
class TestModel(BaseModel):
    field: int = 10

# Optional field
try:
    display(TestModel())
except ValidationError as ex:
    print(ex)

# not nullable field
try:
    TestModel(field=None)
except ValidationError as ex:
    print(ex)

TestModel(field=10)

1 validation error for TestModel
field
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.8/v/int_type


### Optional, Nullable

In [295]:
class TestModel(BaseModel):
    field: int | None = 10

# Optional field
try:
    display(TestModel())
except ValidationError as ex:
    print(ex)

# nullable field
try:
    display(TestModel(field=None))
except ValidationError as ex:
    print(ex)

TestModel(field=10)

TestModel(field=None)

# Inspecting the fields

In [296]:
class Circle(BaseModel):
    center_x: int = 0
    center_y: int = 0
    radius: int = 1
    name: str | None = None

In [297]:
Circle.model_fields

{'center_x': FieldInfo(annotation=int, required=False, default=0),
 'center_y': FieldInfo(annotation=int, required=False, default=0),
 'radius': FieldInfo(annotation=int, required=False, default=1),
 'name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None)}

In [298]:
c1 = Circle(radius=10)
c2 = Circle(name="Circle", center_x=1)

# fields that have been explicitly set on this model instance.
display("c1", c1.model_fields_set)
display("c2", c2.model_fields_set)

'c1'

{'radius'}

'c2'

{'center_x', 'name'}

In [299]:
# model that were assigned from the default values

display("c1", c1.model_fields.keys() - c1.model_fields_set)
display("c2", c2.model_fields.keys() - c2.model_fields_set)



'c1'

{'center_x', 'center_y', 'name'}

'c2'

{'center_y', 'radius'}

__Use Case__

In [300]:
class ApiModel(BaseModel):
    field_1: int = 1
    field_2: int | None = None
    field_3: str
    field_4: str | None = "Python"

In [301]:
m1 = ApiModel(field_3="m1")
m2 = ApiModel(field_1=10, field_3="m2", field_4="Django")
m3 = ApiModel(field_1=10, field_2=20, field_3="m3", field_4="Pydantic")

In [305]:
# This will return the fields
display(m1.model_dump())

# Set fields
display(m1.model_fields_set)

# This will return the set fields
display(m1.model_dump(include=m1.model_fields_set))


{'field_1': 1, 'field_2': None, 'field_3': 'm1', 'field_4': 'Python'}

{'field_3'}

{'field_3': 'm1'}

In [307]:
# This will return the fields
display(m2.model_dump())

# Set fields
display(m2.model_fields_set)

# This will return the set fields
display(m2.model_dump(include=m2.model_fields_set))

{'field_1': 10, 'field_2': None, 'field_3': 'm2', 'field_4': 'Django'}

{'field_1', 'field_3', 'field_4'}

{'field_1': 10, 'field_3': 'm2', 'field_4': 'Django'}

In [308]:
# This will return the fields
display(m3.model_dump())

# Set fields
display(m3.model_fields_set)

# This will return the set fields
display(m3.model_dump(include=m3.model_fields_set))

{'field_1': 10, 'field_2': 20, 'field_3': 'm3', 'field_4': 'Pydantic'}

{'field_1', 'field_2', 'field_3', 'field_4'}

{'field_1': 10, 'field_2': 20, 'field_3': 'm3', 'field_4': 'Pydantic'}

# JSON Schema Generation

Documentation: 
- https://docs.pydantic.dev/latest/api/base_model/#pydantic.BaseModel.model_json_schema

- https://github.com/fbaptiste/pydantic-essentials/blob/main/02%20-%20Basics/10%20-%20JSON%20Schema%20Generation.ipynb

In [310]:
from pprint import pprint

pprint(ApiModel.model_json_schema())

{'properties': {'field_1': {'default': 1,
                            'title': 'Field 1',
                            'type': 'integer'},
                'field_2': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
                            'default': None,
                            'title': 'Field 2'},
                'field_3': {'title': 'Field 3', 'type': 'string'},
                'field_4': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
                            'default': 'Python',
                            'title': 'Field 4'}},
 'required': ['field_3'],
 'title': 'ApiModel',
 'type': 'object'}
